<a href="https://colab.research.google.com/github/edsondamasceno/deep-learning-methods/blob/main/MobileNetV2_%2B_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adapted from Fish classifier & Grad-CAM viz (acc. 99,89%)
Kaggle: https://www.kaggle.com/code/databeru/fish-classifier-grad-cam-viz-acc-99-89

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
from IPython.display import Image, display
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Load and transform the dataset

In [ ]:
train_path = '/content/drive/MyDrive/COVID-19/COVIDx CT ND/Train'
test_path = '/content/drive/MyDrive/COVID-19/COVIDx CT ND/Test'
val_path = '/content/drive/MyDrive/COVID-19/COVIDx CT ND/Val'

# Load the Images with a generator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_images = ImageDataGenerator(rescale=1./255).flow_from_directory(train_path,
                                                                       target_size=(224,224),
                                                                       color_mode='rgb',
                                                                       batch_size=100,
                                                                       class_mode='categorical',
                                                                       shuffle=True)
test_images = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
                                                                      target_size=(224,224),
                                                                      color_mode='rgb',
                                                                      batch_size=100,
                                                                      class_mode='categorical',
                                                                      shuffle=True)
val_images = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path,
                                                                     target_size=(224,224),
                                                                     color_mode='rgb',
                                                                     batch_size=100,
                                                                     class_mode='categorical',
                                                                     shuffle=True)

Found 21024 images belonging to 2 classes.
Found 6007 images belonging to 2 classes.
Found 3007 images belonging to 2 classes.


# Train the model

In [ ]:
# Load the pretained model
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)

pretrained_model.trainable = False

9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(1280, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(1280, activation='relu')(x)

outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [ ]:
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
    ]
)

Epoch 1/50
240/240 [==============================] - 5385s 22s/step - loss: 0.6208 - accuracy: 0.7526 - val_loss: 0.4271 - val_accuracy: 0.8333
Epoch 2/50
240/240 [==============================] - 113s 470ms/step - loss: 0.4321 - accuracy: 0.8283 - val_loss: 0.4288 - val_accuracy: 0.8386
Epoch 3/50
240/240 [==============================] - 119s 494ms/step - loss: 0.3889 - accuracy: 0.8454 - val_loss: 0.3984 - val_accuracy: 0.8520
Epoch 4/50
240/240 [==============================] - 118s 493ms/step - loss: 0.3689 - accuracy: 0.8531 - val_loss: 0.4980 - val_accuracy: 0.8169
Epoch 5/50
240/240 [==============================] - 112s 466ms/step - loss: 0.3387 - accuracy: 0.8684 - val_loss: 0.3642 - val_accuracy: 0.8621
Epoch 6/50
240/240 [==============================] - 111s 463ms/step - loss: 0.3273 - accuracy: 0.8715 - val_loss: 0.4399 - val_accuracy: 0.8300
Epoch 7/50
240/240 [==============================] - 112s 466ms/step - loss: 0.2926 - accuracy: 0.8870 - val_loss: 0.3798 - 

In [ ]:
model.save('/content/drive/MyDrive/COVID-19/MobileNetV2/MobileNetV2.h5')

In [ ]:
import pandas as p
hist_df = pd.DataFrame(history.history)
hist_csv_file = '/content/drive/MyDrive/COVID-19/MobileNetV2/history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

# Visualize the result

In [ ]:
results = model.evaluate(test_images, verbose=0)

print("Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

Test Loss: 0.34468
Test Accuracy: 87.50%


# Feature Extraction

In [ ]:
from keras.models import load_model, Model
import glob
import keras.utils as image

In [ ]:
model = load_model('/content/drive/MyDrive/COVID-19/MobileNetV2/MobileNetV2.h5')

In [ ]:
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer('dense_4').output)
intermediate_layer_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [ ]:
feauture_engg_data = intermediate_layer_model.predict(val_images)
feauture_engg_data = pd.DataFrame(feauture_engg_data)
print('feauture_engg_data shape:', feauture_engg_data.shape)
feauture_engg_data.head(5)  #The features are unnamed now

34/34 [==============================] - 14s 398ms/step
feauture_engg_data shape: (3365, 128)


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.0,0.0,0.000000,4.823138,0.0,4.169002,0.000000,3.205368,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.000000,3.664405,0.0,2.521020,0.000000,0.848568,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.901309,2.448783,0.0,0.0,0.000000,0.0
2,0.0,0.0,1.581135,2.992224,0.0,0.000000,0.173456,0.407309,0.000000,0.0,...,0.0,0.0,0.670956,0.0,3.491685,0.000000,0.0,0.0,0.000000,0.0
3,0.0,0.0,3.275373,3.357286,0.0,1.458176,3.144612,0.000000,0.000000,0.0,...,0.0,0.0,2.473033,0.0,3.978734,0.000000,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.210002,4.296848,0.0,0.000000,0.840242,2.625255,0.149522,0.0,...,0.0,0.0,2.477770,0.0,0.000000,0.000000,0.0,0.0,0.348618,0.0


In [ ]:
def extract_feature_one_image(img_path,intermediate_layer_model,input_img):

    img = image.load_img(img_path, target_size=(input_img, input_img))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)

    features = intermediate_layer_model.predict(img_data)
    features = features.reshape((-1))
    return features

In [ ]:
def create_model():

    input_image = 224
    intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer('dense_4').output)

    return intermediate_layer_model, input_image

In [ ]:
#model_name = 'MobileNetV2'
#database_name = 'Test'

#path_database = '/content/drive/MyDrive/COVID-19/COVID-QU-Ex-V1/' + database_name + '/'
#path_write = '/content/drive/MyDrive/COVID-19/MobileNetV2/' + database_name + '_' + model_name + '.csv'

model_name = 'MobileNetV2'
database_name = 'Covid19-dataset'

path_database = '/content/drive/MyDrive/COVID-19/' + database_name + '/'
path_write = '/content/drive/MyDrive/COVID-19/MobileNetV2/' + database_name + '_' + model_name + '.csv'

intermediate_layer_model, input_img = create_model()
features = []

print('Model created...')

classes = glob.glob(path_database + '*')
cont_classe = 0
features_labels_two_classes = pd.DataFrame()

for classe in classes:
    features = []
    images_names = glob.glob(classe + '/*.*')
    for name in images_names:
        features.append(extract_feature_one_image(name, intermediate_layer_model, input_img))

    features_labels = pd.DataFrame(data=features)
    features_labels['Labels'] = np.uint8(cont_classe)
    features_labels_two_classes = features_labels_two_classes.append(features_labels,
                                                                 ignore_index=True)
    cont_classe+=1
features_labels_two_classes.to_csv(path_write,index=False)

print('Finished!!!!')

Model created...
1/1 [==============================] - 0s 24ms/step


<ipython-input-27-f3293be0e013>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features_labels_two_classes = features_labels_two_classes.append(features_labels,


1/1 [==============================] - 0s 21ms/step


<ipython-input-27-f3293be0e013>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features_labels_two_classes = features_labels_two_classes.append(features_labels,


1/1 [==============================] - 0s 20ms/step
Finished!!!!


<ipython-input-27-f3293be0e013>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features_labels_two_classes = features_labels_two_classes.append(features_labels,


In [ ]:
features_labels_two_classes.shape

(332, 129)